# Neo4j Database Connection and Data Import

This notebook connects to Neo4j database and imports extracted entities and relationships for threat intelligence knowledge graph construction.

## Overview
- **Database Connection**: Connect to Neo4j Aura instance
- **Data Import**: Import entities and relationships from CSV files
- **Graph Analysis**: Analyze the constructed knowledge graph
- **Query Examples**: Sample Cypher queries for threat intelligence analysis

## Prerequisites
- Neo4j Aura instance credentials
- Extracted entities and relationships CSV files
- Python packages: `neo4j`, `python-dotenv`, `pandas`


In [ ]:
# Import required libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from neo4j import GraphDatabase
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("✅ Libraries imported successfully!")


## 1. Database Connection Setup


In [ ]:
# Load environment variables from Neo4j credentials file
load_status = load_dotenv("../Neo4j-3389ae49-Created-2025-08-31.txt")
if load_status is False:
    raise RuntimeError('Environment variables not loaded.')

# Get Neo4j connection details
URI = os.getenv("NEO4J_URI")
USERNAME = os.getenv("NEO4J_USERNAME")
PASSWORD = os.getenv("NEO4J_PASSWORD")
DATABASE = os.getenv("NEO4J_DATABASE", "neo4j")

print(f"🔗 Connecting to Neo4j database...")
print(f"   URI: {URI}")
print(f"   Database: {DATABASE}")
print(f"   Username: {USERNAME}")


In [ ]:
# Create Neo4j driver class
class Neo4jDriver:
    def __init__(self, uri, username, password, database):
        self.uri = uri
        self.username = username
        self.password = password
        self.database = database
        self.driver = None
    
    def connect(self):
        """Connect to Neo4j database"""
        try:
            self.driver = GraphDatabase.driver(self.uri, auth=(self.username, self.password))
            self.driver.verify_connectivity()
            print("✅ Connection established successfully!")
            
            # Get database info
            with self.driver.session(database=self.database) as session:
                result = session.run("CALL dbms.components() YIELD name, versions, edition")
                for record in result:
                    print(f"   Neo4j {record['name']} {record['versions'][0]} ({record['edition']})")
            
            return True
            
        except Exception as e:
            print(f"❌ Connection failed: {e}")
            return False
    
    def close(self):
        """Close Neo4j connection"""
        if self.driver:
            self.driver.close()
            print("✅ Connection closed.")
    
    def run_query(self, query, parameters=None):
        """Run a Cypher query and return all records as list of dicts"""
        try:
            with self.driver.session(database=self.database) as session:
                result = session.run(query, parameters or {})
                # Convert to list of dicts immediately to avoid Result object issues
                records = []
                for record in result:
                    records.append(dict(record))
                return records
        except Exception as e:
            print(f"❌ Query failed: {e}")
            return []

# Create Neo4j driver instance
neo4j_driver = Neo4jDriver(URI, USERNAME, PASSWORD, DATABASE)

# Test connection
if neo4j_driver.connect():
    print("\n✅ Neo4j driver initialized and ready to use!")


## 2. Load Data Files


In [ ]:
# Define data loading class
class DataLoader:
    def __init__(self, data_dir="../data"):
        self.data_dir = data_dir
        self.entities_dir = f"{data_dir}/entities"
        self.relationships_file = f"{data_dir}/relationships_norm.csv"
        
        self.entity_files = {
            'attackers': 'attackers.csv',
            'malware': 'malware.csv',
            'tools': 'tools.csv',
            'vulnerabilities': 'vulnerabilities.csv',
            'files': 'files.csv',
            'urls': 'urls.csv',
            'techniques': 'techniques.csv',
            'hashes': 'hashes.csv',
            'ips': 'ips.csv',
            'devices': 'devices.csv'
        }
        
        self.entity_dataframes = {}
        self.relationships_df = None
        self.total_entities = 0
    
    def load_entities(self):
        """Load entity CSV files"""
        print("📂 Loading entity files...")
        
        for entity_type, filename in self.entity_files.items():
            file_path = f"{self.entities_dir}/{filename}"
            try:
                df = pd.read_csv(file_path)
                self.entity_dataframes[entity_type] = df
                count = len(df)
                self.total_entities += count
                print(f"   ✅ {entity_type}: {count:,} entities")
            except FileNotFoundError:
                print(f"   ⚠️  {entity_type}: File not found")
            except Exception as e:
                print(f"   ❌ {entity_type}: Error loading - {e}")
        
        print(f"\n📊 Total entities loaded: {self.total_entities:,}")
        return self.entity_dataframes
    
    def load_relationships(self):
        """Load relationships CSV file"""
        print(f"\n🔗 Loading relationships...")
        try:
            self.relationships_df = pd.read_csv(self.relationships_file)
            print(f"   ✅ Relationships: {len(self.relationships_df):,} relationships")
            print(f"   ✅ Unique relationship types: {self.relationships_df['relation'].nunique()}")
        except FileNotFoundError:
            print(f"   ❌ Relationships file not found: {self.relationships_file}")
        except Exception as e:
            print(f"   ❌ Error loading relationships: {e}")
        
        return self.relationships_df

# Create data loader instance and load data
data_loader = DataLoader()
entity_dataframes = data_loader.load_entities()
relationships_df = data_loader.load_relationships()


## 3. Import Entities with File-Based Labels


In [ ]:
# Import entities with labels matching CSV file names
def import_entities_with_file_labels(neo4j_driver, entity_dataframes):
    """Import entities using CSV file names as Neo4j labels"""
    print("📥 Importing entities with file-based labels...")
    
    total_imported = 0
    
    for entity_type, df in entity_dataframes.items():
        if len(df) == 0:
            continue
            
        print(f"\n   📂 Importing {entity_type} entities...")
        
        # Use entity_type (file name) as the label
        label = entity_type.title()  # Capitalize first letter
        
        # Create nodes for this entity type
        create_query = f"""
        UNWIND $entities AS entity
        MERGE (n:{label} {{id: entity.id}})
        SET n.name = entity.name,
            n.type = entity.type,
            n.label = '{label}',
            n.imported_at = datetime()
        """
        
        # Prepare data for import
        entities_data = df.to_dict('records')
        
        try:
            # Execute create query
            neo4j_driver.run_query(create_query, {"entities": entities_data})
            
            # Count created nodes
            count_query = f"MATCH (n:{label}) RETURN count(n) as count"
            count_result = neo4j_driver.run_query(count_query)
            if count_result and len(count_result) > 0:
                imported_count = count_result[0]['count']
                print(f"      ✅ Imported {imported_count:,} {entity_type} entities (Label: {label})")
                total_imported += imported_count
            else:
                print(f"      ⚠️  Could not count {entity_type} entities")
                
        except Exception as e:
            print(f"      ❌ Error importing {entity_type}: {e}")
    
    print(f"\n✅ Total entities imported: {total_imported:,}")
    return total_imported

# Run entity import with file-based labels
print("🚀 Starting entity import with file-based labels...")
imported_entities = import_entities_with_file_labels(neo4j_driver, entity_dataframes)


## 4. Verify Entity Import


In [ ]:
# Verify import by checking all labels
print("🔍 Verifying imported entities by label...")
verification_query = """
MATCH (n)
WITH labels(n)[0] as Label, count(n) as Count
RETURN Label, Count
ORDER BY Count DESC
"""

result = neo4j_driver.run_query(verification_query)
if result and len(result) > 0:
    print("   📊 Entities by label:")
    for record in result:
        print(f"      {record['Label']}: {record['Count']:,} entities")
else:
    print("   ❌ Could not verify entities")
